# Imports

In [1]:
from pathlib import Path
import sys
import warnings
import time

abs_path = Path().resolve()
sys.path.append(str(abs_path.parent / "modules"))

import pandas as pd

from files_funcs import *
from docx_funcs import *
from timer_func import *


warnings.filterwarnings("ignore")

start_ = time.time()

# Config

In [2]:
main_dir = abs_path.parent.parent

config_path = abs_path.parent.parent / "config" / "config.yaml"
config = load_yaml(config_path)

data_dir = main_dir / "data"
uc_data_dir = data_dir / "raw_data" / "HMI"
ssts_data_dir = data_dir / "raw_data" / "SSTS"

columns = (
        config["ds_topology"]["main_columns"]
        + list(set(config["ds_topology"]["hmi_subheaders"].values()))
        + config["ds_topology"]["targets"]
)
columns = [col.lower().replace(" ", "_") for col in columns]

# Loading

In [3]:
ds = create_ds(uc_data_dir, ssts_data_dir, columns)
ds

Warning! [Errno 2] No such file or directory: 'C:\\Users\\yytrb\\OneDrive\\Рабочий стол\\atom-compliance-ml\\data\\raw_data\\SSTS\\SSTS-26160.docx'


,id,case_name,full_uc_text,full_ssts_text,other,goal,preconditions,main_scenario,postconditions,differences,description,complience_level
0,26160,Filtering the internet radio stations,[I-26160] Filtering the internet radio statio...,None,Description: Requirenments:\nThe action is dis...,Goal: 'Reorder the Internet Radio stations by ...,Preconditions:\nATOM is connected to the inter...,Main Scenario:\nout_2/out_5 displays the list ...,Postconditions:\nout_2/out_5 displays the inte...,Next words are missing in UC-text:\n- [I-26160...,Next words are missing in SSTS-text:\n,None
1,8604,Source selection for Media output,[I-8604] Source selection for Media output\nD...,Switch music sources\nFunctional Description\n...,Description: Scope: This use case is about sel...,Goal: Enable user selection of multimedia audi...,Preconditions:\nATOM is on \nin_5. Smartphone ...,"Main Scenario:\nUser selects the ""Multimedia"" ...",Postconditions:\nout_2/out_5 shows available m...,Next words are missing in UC-text:\n- [I-8604]...,Next words are missing in SSTS-text:\n Switch:...,None
2,11467,Revoke access to the vehicle from a driver or ...,[I-11467] Revoke access to the vehicle from a...,Users can remotely control the DK deletion thr...,,Goal:\nEnable the Owner to quickly revoke acce...,Preconditions:\nAlice - the owner or the drive...,Main Scenario:\nAlice opens the ATOM applicati...,Postconditions:\nAccess has been successfully ...,Next words are missing in UC-text:\n- [I-11467...,Next words are missing in SSTS-text:\n Users: ...,None
3,8800,Receiving Call Notifications,[I-8800] Receiving Call Notifications\nPrecon...,Receiving Call Notifications\nFunctional Descr...,,,Preconditions\nEntertainment system is ready. ...,Main Scenario:\nAn incoming call is received o...,"Postconditions:\nIf the call is accepted, the ...",Next words are missing in UC-text:\n- [I-8800]...,Next words are missing in SSTS-text:\n Functio...,None
4,31523,Adding Internet Radio to Favorites List,[I-31523] Adding Internet Radio to Favorites ...,Favorite Song operation\nFunctional Descriptio...,Description: Actors:\nCar User \nIVI Triggers:...,Goal: User wants to add favorites internet rad...,Preconditions:\nUser in in the Internet Radio ...,Main Scenario:\nThe user press on the UI inter...,Postconditions:\nThe selected internet radio s...,Next words are missing in UC-text:\n- [I-31523...,Next words are missing in SSTS-text:\n Favorit...,None
5,26161,FM Radio Stations switching,[I-26161] FM Radio Stations switching\nDescri...,Automatic search\nFunctional Description\nSupp...,Description: Requirenments:\nThe action is dis...,,Preconditions:\nUser has selected FM Radio pla...,Main Scenario:\nUser presses the forward/backw...,Postconditions:\nout_5/out_2 shows the station...,Next words are missing in UC-text:\n- [I-26161...,Next words are missing in SSTS-text:\n Automat...,None
6,25957,Mute/unmute the FM Radio playback,[I-25957] Mute/unmute the FM Radio playback\n...,Mute or pause function\nFunctional Description...,Description:,"Goal: ""Users are able to mute/unmute the FM Ra...",Preconditions:\nUser is playing FM Radio,Main Scenario:\n​​​User clicks on Mute/Unmute ...,Postconditions:\nThe user Mutes/Unmutes the FM...,Next words are missing in UC-text:\n- [I-25957...,Next words are missing in SSTS-text:\n Mute: 1...,None
7,30371,ERA Self-diagnosis,[I-30371] ERA Self-diagnosis\nActors:\nERA-Gl...,ERA Self-diagnosis\nFunctional Description\n\n...,Description:\nERA self-diagnosis whether the s...,,,,,Next words are missing in UC-text:\n- [I-30371...,Next words are missing in SSTS-text:\n Functio...,None
8,26771,Turn on and off hotspot via VA,[I-26771] Turn on and off hotspot via VA\n\nD...,\nTurn on and off hotspot\nFunctional Descript...,Description: Scope:\nSWP Android \nVA \nSmartp...,Goal: \nUsers can turn on or off the vehicle h...,,Main scenario:\nThe user asks in_24 Voice. Mic...,,Next words are missing in UC-text:\n- [I-26771...,Next words are missing in SSTS-text:\n Fun

# Target

In [4]:
marked_data = pd.read_excel(data_dir / "raw_data" / "train_data_markup.xlsx")
marked_data = marked_data[["Number", "Complience Level"]]
marked_data = marked_data.rename(columns={"Number": "id", "Complience Level": "complience_level"})
marked_data["target"] = marked_data["complience_level"].fillna("NA").map(config["target_dict"])
marked_data = marked_data.drop("complience_level", axis=1)
marked_data["id"] = marked_data["id"].astype(str)

ds = ds.merge(marked_data, on="id")
ds

,id,case_name,full_uc_text,full_ssts_text,other,goal,preconditions,main_scenario,postconditions,differences,description,complience_level,target
0,26160,Filtering the internet radio stations,[I-26160] Filtering the internet radio statio...,None,Description: Requirenments:\nThe action is dis...,Goal: 'Reorder the Internet Radio stations by ...,Preconditions:\nATOM is connected to the inter...,Main Scenario:\nout_2/out_5 displays the list ...,Postconditions:\nout_2/out_5 displays the inte...,Next words are missing in UC-text:\n- [I-26160...,Next words are missing in SSTS-text:\n,None,1
1,8604,Source selection for Media output,[I-8604] Source selection for Media output\nD...,Switch music sources\nFunctional Description\n...,Description: Scope: This use case is about sel...,Goal: Enable user selection of multimedia audi...,Preconditions:\nATOM is on \nin_5. Smartphone ...,"Main Scenario:\nUser selects the ""Multimedia"" ...",Postconditions:\nout_2/out_5 shows available m...,Next words are missing in UC-text:\n- [I-8604]...,Next words are missing in SSTS-text:\n Switch:...,None,2
2,11467,Revoke access to the vehicle from a driver or ...,[I-11467] Revoke access to the vehicle from a...,Users can remotely control the DK deletion thr...,,Goal:\nEnable the Owner to quickly revoke acce...,Preconditions:\nAlice - the owner or the drive...,Main Scenario:\nAlice opens the ATOM applicati...,Postconditions:\nAccess has been successfully ...,Next words are missing in UC-text:\n- [I-11467...,Next words are missing in SSTS-text:\n Users: ...,None,5
3,8800,Receiving Call Notifications,[I-8800] Receiving Call Notifications\nPrecon...,Receiving Call Notifications\nFunctional Descr...,,,Preconditions\nEntertainment system is ready. ...,Main Scenario:\nAn incoming call is received o...,"Postconditions:\nIf the call is accepted, the ...",Next words are missing in UC-text:\n- [I-8800]...,Next words are missing in SSTS-text:\n Functio...,None,4
4,31523,Adding Internet Radio to Favorites List,[I-31523] Adding Internet Radio to Favorites ...,Favorite Song operation\nFunctional Descriptio...,Description: Actors:\nCar User \nIVI Triggers:...,Goal: User wants to add favorites internet rad...,Preconditions:\nUser in in the Internet Radio ...,Main Scenario:\nThe user press on the UI inter...,Postconditions:\nThe selected internet radio s...,Next words are missing in UC-text:\n- [I-31523...,Next words are missing in SSTS-text:\n Favorit...,None,4
5,26161,FM Radio Stations switching,[I-26161] FM Radio Stations switching\nDescri...,Automatic search\nFunctional Description\nSupp...,Description: Requirenments:\nThe action is dis...,,Preconditions:\nUser has selected FM Radio pla...,Main Scenario:\nUser presses the forward/backw...,Postconditions:\nout_5/out_2 shows the station...,Next words are missing in UC-text:\n- [I-26161...,Next words are missing in SSTS-text:\n Automat...,None,4
6,25957,Mute/unmute the FM Radio playback,[I-25957] Mute/unmute the FM Radio playback\n...,Mute or pause function\nFunctional Description...,Description:,"Goal: ""Users are able to mute/unmute the FM Ra...",Preconditions:\nUser is playing FM Radio,Main Scenario:\n​​​User clicks on Mute/Unmute ...,Postconditions:\nThe user Mutes/Unmutes the FM...,Next words are missing in UC-text:\n- [I-25957...,Next words are missing in SSTS-text:\n Mute: 1...,None,3
7,30371,ERA Self-diagnosis,[I-30371] ERA Self-diagnosis\nActors:\nERA-Gl...,ERA Self-diagnosis\nFunctional Description\n\n...,Description:\nERA self-diagnosis whether the s...,,,,,Next words are missing in UC-text:\n- [I-30371...,Next words are missing in SSTS-text:\n Functio...,None,4
8,26771,Turn on and off hotspot via VA,[I-26771] Turn on and off hotspot via VA\n\nD...,\nTurn on and off hotspot\nFunctional Descript...,Description: Scope:\nSWP Android \nVA \nSmartp...,Goal: \nUsers can turn on or off the vehicle h...,,Main scenario:\nThe user asks in_24 Voice. Mic...,,Next words are missing in UC-text:\n- [I-26771...,Next words are miss

In [5]:
ds.to_csv(data_dir / "ds" / "ds.unl", sep="|", encoding="utf-8", index=False)

In [6]:
timer_val = exec_time(start_, time.time())
timer_val

'Completed for 00:00:01'